In [1]:
from http.client import REQUESTED_RANGE_NOT_SATISFIABLE
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
id_dict = {
    "red light violations": "spqx-js37", "abandoned buildings": "kc9i-wq85",
    "socioeconomic indicators": "kn9c-c2s2", "hardship index": "792q-4jtu",
    "crimes": "ijzp-q8t2", "grocery stores": "4u6w-irs9" }



In [3]:
def pull_data(dataset_id, lim):
    '''
    Given a dataset id, pull out data from Chicago portal API

    Inputs:
      dataset_id: (str) a unique id for one Chicago portal dataset
      limit: (int) number of rows to pull each time, default to None

    Return:
      (Pandas DataFrame) a dataframe pulled from Chicago Portal
    '''
    client = Socrata("data.cityofchicago.org", "goD601SLndI51xcMq1KsnG6np")

    results = client.get(dataset_id, limit = lim)
    return pd.DataFrame.from_records(results)

In [4]:
def gen_tables(id_dictionary, lim):
    dataset_dct = {}
    for dataset_name, url in id_dictionary.items():
        dataset_dct[dataset_name] = pull_data(url, lim)

    return dataset_dct

In [11]:
dct = gen_tables(id_dict, 1000)



red light violations
abandoned buildings
socioeconomic indicators
hardship index
crimes
grocery stores


In [13]:
for key in dct.keys():
    print(key)

red light violations
abandoned buildings
socioeconomic indicators
hardship index
crimes
grocery stores


In [23]:


client = Socrata("data.cityofchicago.org", "goD601SLndI51xcMq1KsnG6np")
results = client.get("ijzp-q8t2", where = f"(primary_type) = 'HOMICIDE'",
                    limit=1000000)

crime_df = pd.DataFrame.from_records(results)


In [24]:

crime_df.columns.unique()

Index(['id', 'case_number', 'date', 'block', 'iucr', 'primary_type',
       'description', 'location_description', 'arrest', 'domestic', 'beat',
       'district', 'ward', 'community_area', 'fbi_code', 'x_coordinate',
       'y_coordinate', 'year', 'updated_on', 'latitude', 'longitude',
       'location'],
      dtype='object')

In [25]:
crime_year_count = crime_df.groupby(by=['community_area', "year"]).size().unstack()


In [ ]:
crime_year_count

In [22]:
dct['grocery stores']['store_name'].unique()

dct['grocery stores'].columns.unique()

Index(['store_name', 'license_id', 'account_number', 'square_feet',
       'size_category', 'address', 'zip_code', 'community_area_name',
       'community_area', 'ward', 'census_tract', 'census_block',
       'x_coordinate', 'y_coordinate', 'latitude', 'longitude', 'location',
       ':@computed_region_rpca_8um6', ':@computed_region_vrxf_vc4k',
       ':@computed_region_6mkv_f3dw', ':@computed_region_bdys_3d7i',
       ':@computed_region_43wa_7qmu', ':@computed_region_awaf_s7ux'],
      dtype='object')

In [33]:
dct['hardship index'].columns.unique()
dct['socioeconomic indicators'].columns.unique()


,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46
2,3,Uptown,3.8,24,8.9,11.8,22.2,35787,20
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6
5,6,Lake View,1.1,11.4,4.7,2.6,17,60058,5
6,7,Lincoln Park,0.8,12.3,5.1,3.6,21.5,71551,2
7,8,Near North Side,1.9,12.9,7,2.5,22.6,88669,1
8,9,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8
9,10,Norwood Park,2,5.4,9,11.5,39.5,32875,21


Grocery stores: 
1. Goals
    a. columns: comm area, if possible comm area name, total groceries stores, num of liquor store, liquor/total gorcery stores ratio, year (??) - Sasha

Crime:
1. Goals
    a. columns: comm area, if possible comm area name, homicide count, avg homicide rate (total homicide/no. of years) - Anthony

socioeconomic indicators
1. Goals
    a. columns: comm_area (make it a number), comma_area name, hardship index, year - Anthony